In [26]:
import fasttext
import json
import random
import glob


# Prepare data from raw

In [35]:
times_to_write_truthy_value = 10
repos = glob.glob("data/scraped/*.json")

with open('data/dataset.train', 'w') as train_dataset:
    with open('data/dataset.test', 'w') as test_dataset:
        for repo in repos:
            with open(repo) as raw_dataset:
                for obj in json.load(raw_dataset):

                    for file in obj['changed_files']:
                        dataset_to_write_into = train_dataset if random.random() < 0.8 else test_dataset
                        obj['title'] = obj['title'].replace('\n', '\t').replace('\r', '\t')
                        obj['body'] = obj['body'].replace('\n', '\t').replace('\r', '\t')
                        file['content'] = file['content'].replace('\n', '\t').replace('\r', '\t')
                        
                        dataset_to_write_into.write(f"__label__included {obj['title']} |_-_| {obj['body']} |_-_| {file['name']} |_-_| {file['content']}\n")
                    for file in obj['unchanged_files']:
                        dataset_to_write_into = train_dataset if random.random() < 0.8 else test_dataset
                        obj['title'] = obj['title'].replace('\n', '\t').replace('\r', '\t')
                        obj['body'] = obj['body'].replace('\n', '\t').replace('\r', '\t')
                        file['content'] = file['content'].replace('\n', '\t').replace('\r', '\t')

                        value = f"__label__excluded {obj['title']} |_-_| {obj['body']} |_-_| {file['name']} |_-_| {file['content']}\n"
                        for _ in range(times_to_write_truthy_value):
                            dataset_to_write_into.write(value)

In [37]:
# Shuffle files
def shuffle_file(filename):
    lines = open(filename).readlines()
    random.shuffle(lines)
    open(filename, 'w').writelines(lines)

shuffle_file("data/dataset.train")
shuffle_file("data/dataset.test")

In [32]:
# Each line should be:
# __label__included Then the whole fucking text
# __label__excluded Then other text here
model = fasttext.train_supervised(input="dataset.train")

In [33]:
model.save_model("model_pr_files_to_change.bin")

In [34]:
model.test("dataset.test")

(8046, 0.9931643052448421, 0.9931643052448421)